In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gradio-3.28.1-py3-none-any.whl (17.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 32.3 MB/s eta 0:00:00
 

In [ ]:
import gradio as gr
import pandas as pd
import sqlite3
from gradio.components import Textbox

In [ ]:
db_path = '/content/drive/MyDrive/Хакатон/startups.db'
table_name = 'companies'

In [ ]:
# connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [ ]:
# Select all data from the table
cursor.execute("SELECT * FROM companies")
rows = cursor.fetchall()

In [ ]:
# Convert the rows to a pandas dataframe
companies_df = pd.DataFrame(rows, columns=['id', 'entity_id', 'name', 'normalized_name', 'category_code', 'status',
                                 'founded_at', 'closed_at', 'domain', 'homepage_url', 'twitter_username',
                                 'logo_url', 'logo_width', 'logo_height', 'short_description',
                                 'overview', 'tag_list', 'state_code', 'city', 'region',
                                 'first_investment_at', 'last_investment_at', 'investment_rounds',
                                 'invested_companies', 'first_funding_at', 'last_funding_at',
                                 'funding_rounds', 'funding_total_usd', 'first_milestone_at',
                                 'last_milestone_at', 'milestones', 'relationships', 'updated_at',
                                 'linkcut', 'category_list', 'crunchbase_profile', 'country', 'founders',
                                 'investors', 'address', 'state', 'seed_profile', 'y_combinator_year',
                                 'city_2023', 'country_2023', 'Industries', 'Employees',
                                 'homepage_url_2023', 'Company_values', 'Org_structure',
                                 'Teams and qty'])

In [216]:
# Define a function that returns the dataframe
def view_data(company, 
              status,
              country, 
              city, 
              funding_rounds, 
              funding_total_usd
              ):
  if company:
    return companies_df[companies_df.normalized_name == str.lower(company)].dropna(axis='columns').head(100)

  if status:
    return companies_df[companies_df.status == float(status)].head(100)

  if country:
    return companies_df[companies_df.country == country].head(100)

  if city:
    return companies_df[companies_df.city == city].head(100)

  if funding_rounds:
    return companies_df[companies_df.funding_rounds >= float(funding_rounds)].head(100)

  if funding_total_usd:
    return companies_df[companies_df.funding_total_usd >= float(funding_total_usd)].head(100)

In [219]:
def view_data(company=None, status=None, country=None, city=None, funding_rounds=None, funding_total_usd=None):
    filtered_df = companies_df.copy()
    
    if company:
        return companies_df[companies_df.normalized_name == str.lower(company)].dropna(axis='columns')

    if status:
        filtered_df = filtered_df[filtered_df.status == float(status)]

    if country:
        filtered_df = filtered_df[filtered_df.country == country]

    if city:
        filtered_df = filtered_df[filtered_df.city == city]

    if funding_rounds:
        filtered_df = filtered_df[filtered_df.funding_rounds >= float(funding_rounds)]

    if funding_total_usd:
        filtered_df = filtered_df[filtered_df.funding_total_usd >= float(funding_total_usd)]

    return filtered_df.head(100)

In [220]:
# Create a Gradio interface to display the dataframe
interface = gr.Interface(fn=view_data, 
                         inputs=[
                             Textbox(label="Name of the object (any case of letters, but the name must be precise)", placeholder='Activision Blizzard'),
                             Textbox(label="Status (0-no information, 1-operating, 2-acquired, 3-ipo, 4-closed)", placeholder='3'),
                             Textbox(label="Country", placeholder='United States'),
                             Textbox(label="City", placeholder='New York'),
                             Textbox(label="Minimum funding rounds", placeholder='10'),
                             Textbox(label="Minimum total funding in usd", placeholder='101001000')
                             ],
                         outputs="dataframe", 
                             title='The joint database of startups', 
                             description="""The database is compiled from the Internet open sources and has > 200K entries. It's still in work. 
                             We'll add more options in further releases.
                             Since it works on google free space it can't load much data, so the output is limited to 100 lines.
                             You can search either by name or by number of other options.
                             You can contact the creators of this application via email loseva.nv@phystech.edu"
                             """,
                             theme='default')

In [221]:
# Launch interface
interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>